In [1]:
import os
import datetime
import pandas as pd
from math import radians, cos, sin, asin, sqrt, pow
from shapely.geometry import MultiPoint
from geopy.distance import great_circle

def distance1(lat_i, lngt_i, lat_j, lngt_j):
    lat_i, lngt_i, lat_j, lngt_j = map(radians, [lat_i, lngt_i, lat_j, lngt_j])
    dlngt = lngt_i - lngt_j
    dlat = lat_i - lat_j
    a = sin(dlat/2)**2 + cos(lat_i) * cos(lat_j) * sin(dlngt/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371
    return c * r * 1000

def computeMean(i, j, type):
    total = 0.0
    if type == 'x':
        for index in range(i, j):
            total += traj[index][0]
    elif type == 'y':
        for index in range(i, j):
            total += traj[index][1]
    return total / (j - i)

def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

def time_trans(q):
    return q.strftime('%Y-%m-%d %H:%M:%S')

def hour_start(q):
    hour = int(q[11:13])
    return hour

def ostart_acc(o, d, o_start, d_start, state):
    if not state == 'stay':
        return datetime.timedelta(minutes=0)
    o_time = datetime.datetime.strptime(o[11:16],'%H:%M')
    d_time = datetime.datetime.strptime(d[11:16],'%H:%M')
    if o_start == d_start:
        return d_time - o_time
    d_start = datetime.datetime.strptime(d[11:13],'%H')
    if d_start - o_time > datetime.timedelta(minutes=0):
        return d_start - o_time
    else:
        return d_start - o_time + datetime.timedelta(days=1)

def dstart_acc(d, o_start, d_start, state):
    if not state == 'stay':
        return datetime.timedelta(minutes=0)
    if o_start == d_start:
        return datetime.timedelta(minutes=0)
    else:
        d_time = datetime.datetime.strptime(d[11:16],'%H:%M')
        d_start = datetime.datetime.strptime(d[11:13],'%H')
        return d_time - d_start

def distance(lat_i, lngt_i, lat_j, lngt_j, state):
    if state == 'stay':
        return 0
    lat_i, lngt_i, lat_j, lngt_j = map(radians, [lat_i, lngt_i, lat_j, lngt_j])
    dlngt = lngt_i - lngt_j
    dlat = lat_i - lat_j
    a = sin(dlat/2)**2 + cos(lat_i) * cos(lat_j) * sin(dlngt/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371
    return c * r * 1000

def ostart_dis(o_acc, d_acc, distance, state):
    if not state == 'move':
        return 0
    if d_acc == datetime.timedelta(minutes=0):
        return distance
    else:
        o_minites = o_acc.day * 1440 + o_acc.hour * 60 + o_acc.minute
        d_minites = d_acc.day * 1440 + d_acc.hour * 60 + d_acc.minute
        total_minutes = o_minites + d_minites
        return double(o_minites) / double(total_minutes) * distance

def dstart_dis(o_acc, d_acc, distance, state):
    if not state == 'move':
        return 0
    if d_acc == datetime.timedelta(minutes=0):
        return 0
    else:
        o_minites = o_acc.day * 1440 + o_acc.hour * 60 + o_acc.minute
        d_minites = d_acc.day * 1440 + d_acc.hour * 60 + d_acc.minute
        total_minutes = o_minites + d_minites
        return double(d_minites) / double(total_minutes) * distance

In [2]:
from sklearn.cluster import DBSCAN
import numpy as np
import matplotlib.pyplot as plt

filename = 'raw_data.csv'
raw_traj = pd.read_csv(filename)
data_segment = pd.DataFrame(columns=['o_lat', 'o_lon', 'o_time', 'd_lat', 'd_lon', 'd_time','cluster_id', 'pid', 'state'])
cluster_center = pd.DataFrame(columns=['pid', 'cluster_id', 'center_lat', 'center_lon'])
cluster_size = pd.DataFrame(columns=['pid', 'cluster_id', 'cluster_size'])
cluster_var = pd.DataFrame(columns=['pid', 'cluster_id', 'cluster_size', 'variance'])
trajectories = pd.DataFrame(columns=['pid', 'trajectory_id', 'path'])
typical_user = raw_traj['pid'].unique()
print(typical_user)

blank_t = datetime.timedelta(days=1)
min_t = datetime.timedelta(minutes=30)
for id in typical_user:
    if id < 10:
        pid = '00' + str(id)
    elif id < 100:
        pid = '0' + str(id)
    else:
        pid = str(id)

    print(pid)
    personal_traj = raw_traj[raw_traj['pid'] == id]
    personal_traj = personal_traj.drop(['pid'], axis=1)
    traj = personal_traj.values.tolist()
    for ti in range(len(traj)):
        traj[ti][2] = datetime.datetime.strptime(traj[ti][2],'%Y-%m-%d %H:%M:%S')

    pointNum = len(traj)
    i = 0
    min_dis = 200
    last_stay = None
    stay_points = []
    stay_start = []
    stay_end = []
    trajectory_count = 0
    while i < pointNum:
        j = i + 1
        while j < pointNum:
            #先排除blank
            if traj[j][2] - traj[j-1][2] > blank_t:
                i = j
                j += 1
                break
            dist = distance1(traj[i][0], traj[i][1], traj[j][0], traj[j][1])
            if dist > min_dis:
                dt = traj[j][2] - traj[i][2]
                if dt > min_t and j - i > 2:
                    stay_points.append([computeMean(i, j, 'x'), computeMean(i, j, 'y'), traj[i][2], traj[j][2], j - i])
                    stay_start.append(i)
                    stay_end.append(j-1)
                i = j
                break
            j += 1
        if j == pointNum:
            break

    coordinates = []
    count = 0
    for s in stay_points:
        coordinates.append([stay_points[count][0], stay_points[count][1]])
        count += 1

    X = np.array(coordinates)
    print(len(X))
    if (len(X)) == 0:
        continue

    kms_per_radian = 6371.0088
    db = DBSCAN(eps=0.5 / kms_per_radian, min_samples=1, metric='haversine').fit(np.radians(X))
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    n_clusters_ = len(set(db.labels_))

    clusters = pd.Series([X[labels == n] for n in range(n_clusters_)])
    centermost_points = clusters.map(get_centermost_point)
    for c in range(n_clusters_):
        cluster_center = cluster_center.append({'pid': pid, 'cluster_id': c-1, 'center_lat': centermost_points[c][0], 'center_lon':  centermost_points[c][0]}, ignore_index=True)

    acc_lat = np.zeros(n_clusters_)
    acc_lat_num = np.zeros(n_clusters_)
    mean_lat = np.zeros(n_clusters_)
    acc_lon = np.zeros(n_clusters_)
    acc_lon_num = np.zeros(n_clusters_)
    mean_lon = np.zeros(n_clusters_)
    variance = np.zeros(n_clusters_)

    i = 0
    cluster_num = 0
    stay_mode = False

    while i < pointNum:
        j = i + 1
        if cluster_num < len(labels) and i == stay_start[cluster_num]:
            j = stay_end[cluster_num]
            if stay_end[cluster_num] + 1 == stay_start[min(cluster_num + 1, len(labels) - 1)]:
                j += 1
            data_segment = data_segment.append({'o_lat': traj[i][0], 'o_lon': traj[i][1], 'o_time': traj[i][2], 'd_lat': traj[j][0], 'd_lon': traj[j][1], 'd_time': traj[j][2], 'cluster_id': labels[cluster_num], 'pid': pid, 'state': 'stay'}, ignore_index=True)
            for si in range(stay_start[cluster_num], stay_end[cluster_num]):
                acc_lat[labels[cluster_num]] += traj[i][0]
                acc_lat_num[labels[cluster_num]] += 1
                acc_lon[labels[cluster_num]] += traj[i][1]
                acc_lon_num[labels[cluster_num]] += 1
            cluster_num += 1
            
        elif traj[min(pointNum-1, j)][2] - traj[i][2] > blank_t:
            data_segment = data_segment.append({'o_lat': traj[i][0], 'o_lon': traj[i][1], 'o_time': traj[i][2], 'd_lat': traj[j][0], 'd_lon': traj[j][1], 'd_time': traj[j][2], 'pid': pid, 'state': 'blank'}, ignore_index=True)

        else:
            #list index in range
            if cluster_num < len(labels):
                while j < stay_start[cluster_num] and traj[j][2] - traj[j-1][2] <= blank_t:
                    j += 1
            else:
                while j < pointNum and traj[j][2] - traj[j-1][2] <= blank_t:
                    j += 1
            if j == pointNum or traj[j][2] - traj[j-1][2] > blank_t:
                    j -= 1
            
            path = str(traj[i][0]) + ',' + str(traj[i][1]) + ',' + str(traj[i][2])
            for tra in range(i+1, j+1):
                path += '; ' + str(traj[tra][0]) + ',' + str(traj[tra][1]) + ',' + str(traj[tra][2])
            trajectories = trajectories.append({'pid': pid, 'trajectory_id': trajectory_count, 'path': path}, ignore_index=True)
            trajectory_count += 1

            if traj[j][2] - traj[j-1][2] > blank_t:
                print('error move')
                print('i: ' + str(i) + '  j: ' + str(j))
                print(traj[j-1][2])
                print(traj[j][2])
                if cluster_num < len(labels):
                    print('next stay ' + str(stay_start[cluster_num]))
                else:
                    print('exceed')
            data_segment = data_segment.append({'o_lat': traj[i][0], 'o_lon': traj[i][1], 'o_time': traj[i][2], 'd_lat': traj[j][0], 'd_lon': traj[j][1], 'd_time': traj[j][2], 'pid': pid, 'state': 'move'}, ignore_index=True)
        i = j
        #last point recorded
        if j == pointNum - 1:
            break

    for clu in range(n_clusters_):
        mean_lat[clu] = acc_lat[clu] / acc_lat_num[clu]
        mean_lon[clu] = acc_lon[clu] / acc_lon_num[clu]
        if clu < n_clusters_ - 1:
            cluster_size = cluster_size.append({'pid': pid, 'cluster_id': clu, 'cluster_size':  acc_lat_num[clu]}, ignore_index=True)
        else:
            cluster_size = cluster_size.append({'pid': pid, 'cluster_id': -1, 'cluster_size':  acc_lat_num[clu]}, ignore_index=True)

    for stay in range(len(coordinates)):
        variance[labels[stay]] += pow(distance1(mean_lat[labels[stay]], mean_lon[labels[stay]], coordinates[stay][0], coordinates[stay][1]), 2)

    for clu in range(n_clusters_):
        variance[clu] /= acc_lat_num[clu]
        if clu < n_clusters_ - 1:
            cluster_var = cluster_var.append({'pid': pid, 'cluster_id': clu, 'cluster_size':  acc_lat_num[clu], 'variance': variance[clu]}, ignore_index=True)
        else:
            cluster_var = cluster_var.append({'pid': pid, 'cluster_id': -1, 'cluster_size':  acc_lat_num[clu], 'variance': variance[clu]}, ignore_index=True)



[  0   2   3   4   5  10  13  14  15  17  20  22  24  25  30  34  36  37
  38  39  41  42  43  44  46  51  52  55  56  58  59  62  65  66  67  68
  71  74  78  82  83  84  85  91  92  95  96 101 104 111 112 114 115 122
 125 126 128 133 134 140 142 144 153 163 167 168 174]
000
470
002
344
003
1040
004
1301
005
190
010
186
013
163
014
451
015
125
017
861
020
134
022
432
024
275
025
655
030
926
034
228
036
211
037
186
038
311
039
485
041
552
042
212
043
87
044
107
046
30
051
87
052
244
055
30
056
22
058
20
059
12
062
427
065
155
066
41
067
178
068
859
071
100
074
69
078
94
082
145
083
42
084
360
085
555
091
80
092
185
095
36
096
149
101
77
104
205
111
54
112
226
114
18
115
157
122
40
125
78
126
361
128
1897
133
9
134
64
140
379
142
208
144
681
153
3003
163
1234
167
602
168
121
174
95


In [3]:
data_segment['o_time'] = data_segment['o_time'].apply(time_trans)
data_segment['d_time'] = data_segment['d_time'].apply(time_trans)
data_segment['ostart_h'] = data_segment['o_time'].apply(hour_start)
data_segment['dstart_h'] = data_segment['d_time'].apply(hour_start)
data_segment['ostart_acc'] = data_segment.apply(lambda x: ostart_acc(x['o_time'], x['d_time'], x['ostart_h'], x['dstart_h'], x['state']), axis = 1)
data_segment['dstart_acc'] = data_segment.apply(lambda x: dstart_acc(x['d_time'], x['ostart_h'], x['dstart_h'], x['state']), axis = 1)
data_segment.dropna(subset=['cluster_id']).sort_values(by=['pid', 'cluster_id', 'o_time']).to_csv('stay_raw.csv', index=False)

o_stay = data_segment[['pid', 'cluster_id', 'ostart_h', 'ostart_acc']]
o_stay.columns = ['pid', 'cluster_id', 'hour', 'stay_time']
d_stay = data_segment[['pid', 'cluster_id', 'dstart_h', 'dstart_acc']]
d_stay.columns = ['pid', 'cluster_id', 'hour', 'stay_time']
stay_acc = pd.concat([o_stay, d_stay]).dropna(subset=['cluster_id'])
stay_acc = stay_acc.sort_values(by=['pid', 'cluster_id', 'hour'])
stay_acc = stay_acc.groupby(['pid', 'cluster_id', 'hour']).sum('stay_time')


stay_cluster = stay_acc.groupby(['pid', 'cluster_id']).sum('stay_time')


data_segment['distance'] = data_segment.apply(lambda x: distance(x['o_lat'], x['o_lon'], x['d_lat'], x['d_lon'], x['state']), axis=1)
data_segment['ostart_dis'] = data_segment.apply(lambda x: ostart_dis(x['ostart_acc'], x['dstart_acc'], x['distance'], x['state']), axis=1)
data_segment['dstart_dis'] = data_segment.apply(lambda x: dstart_dis(x['ostart_acc'], x['dstart_acc'], x['distance'], x['state']), axis=1)
dis_acc = pd.DataFrame(columns=['pid', 'hour', 'distance'])
o_dis = data_segment[['pid', 'ostart_h', 'ostart_dis']]
o_dis.columns = ['pid', 'hour', 'distance']
d_dis = data_segment[['pid', 'dstart_h', 'dstart_dis']]
d_dis.columns = ['pid', 'hour', 'distance']
dis_acc = pd.concat([o_dis, d_dis])
dis_acc = dis_acc.groupby(['pid', 'hour']).sum('distance')
